In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras
import mpld3

import rain

Using TensorFlow backend.


In [2]:
mpld3.disable_notebook()
#mpld3.enable_notebook()

In [3]:
FREQUENCY = 0.5

df = pd.read_csv(rain.toRelPath("data/df-cache.csv"))
df = df.iloc[::(1000 * FREQUENCY // 500), :]
df.reset_index(inplace = True)

mid = df["mid"]
mid.fillna(method = "bfill", inplace = True)
print(mid)

0         7436.995
1         7437.015
2         7437.015
3         7437.015
4         7437.015
5         7437.030
6         7436.995
7         7436.980
8         7437.010
9         7436.970
10        7436.985
11        7436.970
12        7436.965
13        7436.955
14        7436.955
15        7436.955
16        7436.985
17        7436.995
18        7437.040
19        7437.040
20        7437.040
21        7437.370
22        7437.370
23        7437.370
24        7437.380
25        7437.385
26        7437.385
27        7437.775
28        7437.365
29        7437.365
            ...   
734329    7385.135
734330    7385.150
734331    7387.795
734332    7391.740
734333    7392.775
734334    7391.615
734335    7391.860
734336    7391.850
734337    7392.100
734338    7392.090
734339    7392.135
734340    7392.090
734341    7392.105
734342    7392.095
734343    7392.105
734344    7392.100
734345    7392.100
734346    7392.100
734347    7390.010
734348    7391.995
734349    7391.995
734350    73

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(512, input_shape = (50,)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(512))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(512))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(1))
model.add(keras.layers.Activation("sigmoid"))
model.compile(loss = "binary_crossentropy", 
	optimizer = keras.optimizers.adam(lr = 0.0003))